<a href="https://colab.research.google.com/github/jdeept/Financial-News/blob/main/Personalized_Offer_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

jnyandeep0506_amex_final_lap_path = kagglehub.dataset_download('jnyandeep0506/amex-final-lap')

print('Data source import complete.')


In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import gc
from pathlib import Path
from sklearn.model_selection import GroupKFold

In [ ]:
DATA_DIR = Path('/kaggle/input/amex-final-lap')
WORK_DIR = Path('/kaggle/working/')

# Load only selected columns to reduce memory footprint
feature_cols = [f"f{i}" for i in range(1, 201)]  # adjust based on memory budget
base_cols = ['id1', 'id2', 'id3', 'id4', 'id5', 'y']

train_df = pd.read_parquet(DATA_DIR / 'train_data.parquet', columns=base_cols + feature_cols)
test_df = pd.read_parquet(DATA_DIR / 'test_data.parquet', columns=[c for c in base_cols if c != 'y'] + feature_cols)
offer_df = pd.read_parquet(DATA_DIR / 'offer_metadata (2).parquet')
event_df = pd.read_parquet(DATA_DIR / 'add_event (1).parquet')
trans_df = pd.read_parquet(DATA_DIR / 'add_trans (2).parquet')


In [ ]:
for df in [train_df, test_df, trans_df]:
    for col in df.select_dtypes(include='float64').columns:
        df[col] = df[col].astype('float32')
    for col in df.select_dtypes(include='int64').columns:
        df[col] = df[col].astype('int32')


In [ ]:
# --- DEBUG: Print data types of all files ---
print("\n📊 train_df.dtypes:")
print(train_df.dtypes)

print("\n📊 test_df.dtypes:")
print(test_df.dtypes)

print("\n📊 offer_df.dtypes:")
print(offer_df.dtypes)

print("\n📊 event_df.dtypes:")
print(event_df.dtypes)

print("\n📊 trans_df.dtypes:")
print(trans_df.dtypes)

# Optional: After preprocessing
print("\n📊 combined_df.dtypes (after merge):")
print(combined_df.dtypes)



📊 train_df.dtypes:
id1     object
id2     object
id3     object
id4     object
id5     object
         ...  
f196    object
f197    object
f198    object
f199    object
f200    object
Length: 206, dtype: object

📊 test_df.dtypes:
id1     object
id2     object
id3     object
id4     object
id5     object
         ...  
f196    object
f197    object
f198    object
f199    object
f200    object
Length: 205, dtype: object

📊 offer_df.dtypes:
id3       int32
id9      object
f375      int32
f376    float64
f377     object
id10     object
id11     object
f378     object
f374     object
id8      object
id12     object
id13     object
dtype: object

📊 event_df.dtypes:
id2     int32
id3    object
id6    object
id4    object
id7    object
dtype: object

📊 trans_df.dtypes:
id2       int32
f367    float32
f368     object
f369     object
f370     object
f371     object
f372     object
id8      object
f374     object
dtype: object

📊 combined_df.dtypes (after merge):
id1                          obj

In [ ]:
# --- 2. PREPROCESSING ---

# Convert id columns to numeric as needed
combined_df['id2'] = pd.to_numeric(combined_df['id2'], errors='coerce').astype('int32')
combined_df['id3'] = pd.to_numeric(combined_df['id3'], errors='coerce').astype('int32')
offer_df['id3'] = pd.to_numeric(offer_df['id3'], errors='coerce').astype('int32')
event_df['id2'] = pd.to_numeric(event_df['id2'], errors='coerce').astype('int32')
trans_df['id2'] = pd.to_numeric(trans_df['id2'], errors='coerce').astype('int32')
train_df['is_train'] = 1
test_df['is_train'] = 0
test_df['y'] = np.nan  # placeholder for concat
combined_df = pd.concat([train_df, test_df], ignore_index=True)
del train_df, test_df; gc.collect()

# Convert datetime columns
combined_df['id4'] = pd.to_datetime(combined_df['id4'], errors='coerce')
combined_df['id5'] = pd.to_datetime(combined_df['id5'], errors='coerce')
offer_df['id12'] = pd.to_datetime(offer_df['id12'], errors='coerce')
offer_df['id13'] = pd.to_datetime(offer_df['id13'], errors='coerce')
trans_df['f370'] = pd.to_datetime(trans_df['f370'], errors='coerce')
combined_df['id4'] = pd.to_datetime(combined_df['id4'], errors='coerce')
combined_df['id5'] = pd.to_datetime(combined_df['id5'], errors='coerce')
offer_df['id12'] = pd.to_datetime(offer_df['id12'], errors='coerce')
offer_df['id13'] = pd.to_datetime(offer_df['id13'], errors='coerce')

# Ensure id3 is of same type before merging
combined_df['id3'] = pd.to_numeric(combined_df['id3'], errors='coerce')
offer_df['id3'] = pd.to_numeric(offer_df['id3'], errors='coerce')
combined_df['id3'] = combined_df['id3'].astype('int32')
offer_df['id3'] = offer_df['id3'].astype('int32')

# Merge offer metadata
combined_df = combined_df.merge(offer_df, on='id3', how='left')

# Fix datetime columns post-merge
combined_df['id12'] = pd.to_datetime(combined_df['id12'], errors='coerce')
combined_df['id13'] = pd.to_datetime(combined_df['id13'], errors='coerce')
del offer_df; gc.collect()

# Sort for time-based features
combined_df = combined_df.sort_values(by=['id2', 'id4']).reset_index(drop=True)

# ✅ Final sort enforcement
combined_df = combined_df.sort_values(by=['id2', 'id4']).reset_index(drop=True)
trans_df = trans_df.sort_values(by=['id2', 'f370']).reset_index(drop=True)


# Time-based features
combined_df['time_since_last_impression'] = combined_df.groupby('id2')['id4'].diff().dt.total_seconds()
combined_df['impression_rank'] = combined_df.groupby('id2').cumcount() + 1
combined_df['times_seen_offer'] = combined_df.groupby(['id2', 'id3']).cumcount()
combined_df['offer_duration_days'] = (combined_df['id13'] - combined_df['id12']).dt.days
combined_df['dayofweek'] = combined_df['id4'].dt.dayofweek
combined_df['hour'] = combined_df['id4'].dt.hour

# Ensure id2 types match before merging on it
combined_df['id2'] = pd.to_numeric(combined_df['id2'], errors='coerce').astype('int32')
event_df['id2'] = pd.to_numeric(event_df['id2'], errors='coerce').astype('int32')

# Event features
event_agg = event_df.groupby('id2')['id3'].count().reset_index()
event_agg.columns = ['id2', 'total_impressions']
combined_df = combined_df.merge(event_agg, on='id2', how='left')
del event_df, event_agg; gc.collect()

# Ensure id2 type is consistent in trans_df
trans_df['id2'] = pd.to_numeric(trans_df['id2'], errors='coerce').astype('int32')

# Transaction features
trans_df['f370'] = pd.to_datetime(trans_df['f370'], errors='coerce')
trans_agg = trans_df.groupby('id2')['f367'].agg(['mean', 'sum', 'count', 'std']).reset_index()
trans_agg.columns = ['id2', 'avg_spend', 'total_spend', 'tx_count', 'spend_std']
combined_df = combined_df.merge(trans_agg, on='id2', how='left')
combined_df = combined_df.sort_values(by=['id2', 'id4'])
trans_df = trans_df.sort_values(by=['id2', 'f370']).reset_index(drop=True)
combined_df = pd.merge_asof(
    combined_df,
    trans_df[['id2', 'f370', 'f367']],
    left_on='id4', right_on='f370', by='id2', direction='backward')
combined_df['time_since_last_tx'] = (combined_df['id4'] - combined_df['f370']).dt.total_seconds()
combined_df.drop(columns=['f370'], inplace=True)
del trans_df, trans_agg; gc.collect()

ValueError: left keys must be sorted

In [ ]:
!rm -rf /kaggle/working/*
